In [1]:
# For Python2.6 compatibility
from __future__ import print_function

import pretty_midi
import numpy as np
# For plotting
import matplotlib.pyplot as plt
%matplotlib inline
# For putting audio in the notebook
import IPython.display
import copy

inputtedMidi = pretty_midi.PrettyMIDI('eminor.mid')
times, tempo_changes = inputtedMidi.get_tempo_changes()

pitchInventory = [0 for i in range(12)]
for instrument in inputtedMidi.instruments:
    for note in instrument.notes:
        pitchInventory[note.pitch%12] += 1

copiedPitchCount = copy.deepcopy(pitchInventory)
copiedPitchCount.sort()
copiedPitchCount.reverse()
cardinality = len(copiedPitchCount)
for i in range(len(copiedPitchCount)):
    if copiedPitchCount[i] == 0:
        cardinality = i
        break
threshold = copiedPitchCount[6]
for i in range(7, len(copiedPitchCount)):
    if copiedPitchCount[i] < threshold:
        threshold = copiedPitchCount[i]
        break

        
adjustedPitchInventory = [(i if i > threshold else 0) for i in pitchInventory]
majorSteps = [2, 2, 1, 2, 2, 2]
harmonicMinorSteps = [2, 1, 2, 2, 1, 3]
keyType = -1
tonic = -1

In [2]:
for i in range(0, 12):
    if keyType != -1:
        break
    
    if adjustedPitchInventory[i] == 0:
        continue
    
    fits = True
    currentPitch = i
    for j in range(min(len(majorSteps), cardinality-1)):
        currentPitch = (currentPitch + majorSteps[j]) % 12
        if not adjustedPitchInventory[currentPitch]:
            fits = False
            break
    if fits:
        keyType = 0
        tonic = i
        break
    
    fits = True
    currentPitch = i
    for j in range(min(len(majorSteps), cardinality-1)):
        currentPitch = (currentPitch + harmonicMinorSteps[j]) % 12
        if not adjustedPitchInventory[currentPitch]:
            fits = False
            break
    if fits:
        keyType = 1
        tonic = i
        break

print(keyType)
print(tonic)

1
4


In [53]:
import pretty_midi
import numpy as np
from functools import cmp_to_key
import heapq
import bisect

def get_examples(path, tonic, mode):
    input_midi = pretty_midi.PrettyMIDI(path)
    print(path)
    
    times, tempo_changes = input_midi.get_tempo_changes() 
    
    midi_notes = input_midi.instruments[0].notes;
    
    beats = input_midi.get_beats()
    beat_ticks = [input_midi.time_to_tick(i) for i in beats]
    
    def compare(x, y):
        result = False
        if (abs(x.start - y.start) < 0.000001):
            result = -1 if (x.pitch > y.pitch) else 1
        else:
            result = -1 if (x.start < y.start) else 1
        return result
    midi_notes.sort(key=cmp_to_key(compare))
    
    start_end_events = []
    for i in range(len(midi_notes)):
        start_end_events.append((input_midi.time_to_tick(midi_notes[i].start), midi_notes[i].pitch, i+1))
        start_end_events.append((input_midi.time_to_tick(midi_notes[i].end), midi_notes[i].pitch, -(i+1)))
        
    def compareNoteEvents(x, y):
        result = 0
        # sort by pitch
        if x[0] == y[0]:
            result = -1 if (x[1] > y[1]) else 1
        else:
            result = -1 if (x[0] < y[0]) else 1
        return result
    
    start_end_events.sort(key=cmp_to_key(compareNoteEvents))
    
    highest_pitch_heap = []
    current_time = start_end_events[0][0]
    current_melody = start_end_events[0][1]
    current_notes = set()
    heapq.heappush(highest_pitch_heap, (-start_end_events[0][1], start_end_events[0][2]))
    current_notes.add(start_end_events[0][2])
    
    melody = []
    accompaniment = []
    
    for i in range(1, len(start_end_events)):
        current_event = start_end_events[i]
        
        if current_time != current_event[0]: # new time, append to melody and accompaniment
            # get rid of outdated
            
            while highest_pitch_heap and not highest_pitch_heap[0][1] in current_notes:
                heapq.heappop(highest_pitch_heap)
            
            if not highest_pitch_heap:
                if current_event[2] > 0:
                    current_notes.add(current_event[2])
                    heapq.heappush(highest_pitch_heap, (-current_event[1], current_event[2]))
                else:
                    current_notes.remove(-current_event[2])
                continue
            
            melody_id = highest_pitch_heap[0][1]
            melody.append((current_time, -highest_pitch_heap[0][0]))
            
            chord = []
            for j in current_notes:
                if j != melody_id:
                    chord.append(midi_notes[j-1].pitch)
            accompaniment.append((current_time, tuple(chord)))
            current_time = current_event[0]
        
        if current_event[2] > 0:
            current_notes.add(current_event[2])
            heapq.heappush(highest_pitch_heap, (-current_event[1], current_event[2]))
        else:
            current_notes.remove(-current_event[2])
    
#     print(melody)
#     print(accompaniment)

    melody_pitches = [[0 for i in range(12)] for j in range(len(beat_ticks))]
    accompaniment_pitches = [[0 for i in range(12)] for j in range(len(beat_ticks))]
    
    for note in melody:
        melody_pitches[bisect.bisect_right(beat_ticks, note[0])-1][(note[1] - tonic + 12) % 12] = 1
    
    for chord in accompaniment:
        for pitch in chord[1]:
            accompaniment_pitches[bisect.bisect_right(beat_ticks, chord[0])-1][(pitch - tonic + 12) % 12] = 1
    
    melody_pitches = np.asarray(melody_pitches)
    accompaniment_pitches = np.asarray(accompaniment_pitches)
    
    
    # remove empty pitch inventories at end
    empty_pitch_inventory = np.zeros((12,), dtype='int64')
    
    removed_melody_pitches = []
    removed_accompaniment_pitches = []
    
    for i in range(len(accompaniment_pitches)-1, -1, -1):
        if np.array_equal(accompaniment_pitches[i], empty_pitch_inventory):
            continue
        removed_melody_pitches.append(melody_pitches[i])
        removed_accompaniment_pitches.append(accompaniment_pitches[i])
    
    removed_melody_pitches = np.asarray(removed_melody_pitches)
    removed_accompaniment_pitches = np.asarray(removed_accompaniment_pitches)
    
#     last_melody = len(melody_pitches)
#     last_accompaniment = len(accompaniment_pitches)
#     for i in range(len(melody_pitches)-1, -1, -1):
#         if np.array_equal(melody_pitches[i], empty_pitch_inventory):
#             last_melody = i
#         else:
#             break
    
#     for i in range(len(accompaniment_pitches)-1, -1, -1):
#         if np.array_equal(accompaniment_pitches[i], empty_pitch_inventory):
#             last_accompaniment = i
#         else:
#             break
    
#     cutoff = min(last_melody, last_accompaniment)
#     melody_pitches = melody_pitches[:cutoff]
#     accompaniment_pitches = accompaniment_pitches[:cutoff]

#     mode_one_hot = np.zeros((accompaniment))
    examples = np.concatenate([removed_accompaniment_pitches[:-1], removed_melody_pitches[1:], removed_accompaniment_pitches[1:]], axis=1)
    
    # remove duplicates
    examples = np.vstack(list({tuple(row) for row in examples}))
    
    # attach mode one hot encoding
    mode_one_hot = np.zeros((len(examples), 2))
    mode_one_hot[:, mode] = 1
    examples = np.concatenate([mode_one_hot, examples], axis=1)

    return examples
    
    
test = [5, 9, 14, 15, 19]
print(bisect.bisect_left(test, 16))
    
#     notes_heap = []
#     current_time = midi_notes[0].start
#     for note in midi_notes:
#         current_time = note.start
#         heappush((-note.pitch, note.end))
        
#         highest = heappop(h)
#         while(highest[1].end < )
#         print((note.start, note.end, note.pitch))

# chpn-p15_format0.mid

# get_examples('chpn-p15_format0.mid', 1, 1)

4


In [38]:
import os

all_examples = None
path = os.fsencode("")
for entry in os.scandir(path):
    file_name = entry.name.decode("utf-8")
    if not file_name.endswith(".mid") or file_name == "a0_3b.mid" or file_name == "a0_1.mid" or file_name == "b1_2b.mid":
        continue
    
    new_examples = get_examples("training/" + file_name, ord(file_name[0])-ord('a'), int(file_name[1]))
    if all_examples is None:
        all_examples = new_examples
    else:
        all_examples = np.vstack((all_examples, new_examples))
    print(file_name)
    print(all_examples.shape)

all_examples = np.vstack(list({tuple(row) for row in all_examples}))
print(all_examples.shape)
np.savetxt("all_examples.csv", all_examples, delimiter=",")

training/d0_2b.mid
d0_2b.mid
(112, 38)
training/b1_3b.mid
b1_3b.mid
(458, 38)
training/h0_2m.mid
h0_2m.mid
(499, 38)
training/a1_5b.mid
a1_5b.mid
(935, 38)
training/k1_1.mid
k1_1.mid
(1074, 38)
training/g1_1.mid
g1_1.mid
(1169, 38)
training/k0_8m.mid
k0_8m.mid
(1314, 38)
training/e1_1.mid
e1_1.mid
(1388, 38)
training/i1_1.mid
i1_1.mid
(1533, 38)
training/k0_9m.mid
k0_9m.mid
(1619, 38)
training/c0_1.mid
c0_1.mid
(1683, 38)
training/h0_3m.mid
h0_3m.mid
(1757, 38)
training/a1_4b.mid
a1_4b.mid
(2366, 38)
training/k0_5m.mid
k0_5m.mid
(2405, 38)
training/d0_3b.mid
d0_3b.mid
(2649, 38)
training/c0_3m.mid
c0_3m.mid
(2767, 38)
training/g0_1.mid
g0_1.mid
(2825, 38)
training/k0_1.mid
k0_1.mid
(2949, 38)
training/a0_4m.mid
a0_4m.mid
(2995, 38)
training/a1_1.mid
a1_1.mid
(3026, 38)
training/d0_4b.mid
d0_4b.mid
(3154, 38)
training/j0_3m.mid
j0_3m.mid
(3229, 38)
training/a1_3b.mid
a1_3b.mid
(3339, 38)
training/a1_2b.mid
a1_2b.mid
(3514, 38)
training/f0_2b.mid
f0_2b.mid
(3536, 38)
training/j0_2m.mid
j

In [285]:
import pretty_midi
import numpy as np
from functools import cmp_to_key
import heapq
import bisect
import random

def get_accompaniment(name, tonic, mode):
    input_midi = pretty_midi.PrettyMIDI(name)
    
    midi_notes = input_midi.instruments[0].notes;
    
    beats = input_midi.get_beats()
    beat_ticks = [input_midi.time_to_tick(i) for i in beats]
    accompaniment = np.zeros((len(beat_ticks), 12), dtype='int64')
    
    def compare(x, y):
        result = False
        if (abs(x.start - y.start) < 0.000001):
            result = -1 if (x.pitch > y.pitch) else 1
        else:
            result = -1 if (x.start < y.start) else 1
        return result
    midi_notes.sort(key=cmp_to_key(compare))
    
#     start_end_events = []
#     for i in range(len(midi_notes)):
#         start_end_events.append((input_midi.time_to_tick(midi_notes[i].start), midi_notes[i].pitch, i+1))
#         start_end_events.append((input_midi.time_to_tick(midi_notes[i].end), midi_notes[i].pitch, i+1))
    
    pitches_at_beat = np.zeros((len(beat_ticks), 12), dtype='int64')
    for i in midi_notes:
        lBeat = bisect.bisect_right(beat_ticks, input_midi.time_to_tick(i.start)) - 1
        rBeat = bisect.bisect_left(beat_ticks, input_midi.time_to_tick(i.end)) - 1
        pitches_at_beat[lBeat:rBeat+1, (i.pitch - tonic + 12) % 12] = 1

    
    scales = np.asarray([[0, 2, 4, 5, 7, 9, 11], # major
                        [0, 2, 3, 5, 7, 8, 11]]) # harmonic minor
    possible_secondary_dominants = [
            [
                [7, 11, 2, 5],
                [9, 1, 4, 7],
                [11, 3, 6, 9],
                [0, 4, 7, 10],
                [2, 6, 9, 0],
                [4, 8, 11, 2],
                None
            ],
            [
                [7, 11, 2, 5],
                None,
                None,
                [0, 4, 7, 10],
                [2, 6, 9, 0],
                [3, 7, 10, 1],
                None
            ]
        ]
    possible_secondary_leading_tones = [
        [
            [11, 2, 5, 9],
            [1, 4, 7, 10],
            [3, 6, 9, 0],
            [4, 7, 10, 2],
            [6, 9, 0, 4],
            [8, 11, 2, 5],
            None
        ],
        [
            [11, 2, 5, 8],
            None,
            None,
            [4, 7, 10, 1],
            [6, 9, 0, 4],
            [7, 10, 1, 5],
            None
        ]
    ]
    chord_choices = ["" for i in range(len(pitches_at_beat))]
    downbeat_ticks = [input_midi.time_to_tick(i) for i in input_midi.get_downbeats()]
    actual_i = len(pitches_at_beat)-1
    for i in range(len(pitches_at_beat)-1, -1, -1):
        if i != actual_i:
            continue
        
        upcoming_downbeat = (bisect.bisect_right(downbeat_ticks, beat_ticks[i]) - 1) * 4
        
        space_choice_pool = [1]
        if i - upcoming_downbeat + 1 == 4:
            space_choice_pool.append(4)
        
        if i - upcoming_downbeat + 1 >= 2:
            space_choice_pool.append(2)
    
        beat_space = random.choice(space_choice_pool)
        
        pitch_inventory = np.zeros((12,), dtype='int64')
        for j in range(i - beat_space + 1, i+1):
            for k in range(len(pitches_at_beat[j])):
                pitch_inventory[k] |= pitches_at_beat[j][k]
                
        chord_choice_pool = []

        # triads and sevenths
        for root in range(7):
            for extension in [0, 2, 4]:
                if pitch_inventory[scales[mode][(root + extension) % 7] % 12] == 1:
                    chord_choice_pool.append("di" + str(root))
                    if (i < len(chord_choices) - 1 and (root + 4) % 7 == chord_choices[i+1]):
                        for i in range(2):
                            chord_choice_pool.append("di" + str(root))
        
        # secondary dominants
        if i < len(pitches_at_beat) - 1 and possible_secondary_dominants[mode][int(chord_choices[i+1][-1])]:
            for j in possible_secondary_dominants[mode][int(chord_choices[i+1][-1])]:
                if pitch_inventory[j] == 1:
                    chord_choice_pool.append("2d"+chord_choices[i+1][-1])
        # secondary leading tones
        if i < len(pitches_at_beat) - 1 and possible_secondary_leading_tones[mode][int(chord_choices[i+1][-1])]:
            for j in possible_secondary_leading_tones[mode][int(chord_choices[i+1][-1])]:
                if pitch_inventory[j] == 1:
                    chord_choice_pool.append("2l"+chord_choices[i+1][-1])
        
        final_choice = random.choice(chord_choice_pool)
        for j in range(i-beat_space+1, i+1):
            chord_choices[j] = final_choice
        actual_i -= beat_space
    
    pitch_list = [[] for i in range(len(accompaniment))]
    for i in range(len(chord_choices)):
        if chord_choices[i][:2] == "di":
            for j in [0, 2, 4]:
                accompaniment[i][(scales[mode][(int(chord_choices[i][-1])+j) % 7] + tonic) % 12] = 1
                pitch_list[i].append((scales[mode][(int(chord_choices[i][-1])+j) % 7] + tonic) % 12)
            if random.choice([0, 1, 2]) == 2:
                accompaniment[i][(scales[mode][(int(chord_choices[i][-1])+6) % 7] + tonic) % 12] = 1
                pitch_list[i].append((scales[mode][(int(chord_choices[i][-1])+6) % 7] + tonic) % 12)
        elif chord_choices[i][:2] == "2d":
            for j in possible_secondary_dominants[mode][int(chord_choices[i][-1])]:
                accompaniment[i][(j+tonic)%12] = 1
                pitch_list[i].append((j+tonic)%12)
        elif chord_choices[i][:2] == "2l":
            for j in possible_secondary_leading_tones[mode][int(chord_choices[i][-1])]:
                accompaniment[i][(j+tonic)%12] = 1
                pitch_list[i].append((j+tonic)%12)
    
    voices = [[0 for j in range(len(pitch_list))] for i in range(4)]
    if (len(pitch_list[0]) == 3):
        pitch_list[0].append(pitch_list[0][0])
        if pitch_list[0][2] > pitch_list[0][0]:
            pitch_list[0][3] += 12
        else:
            pitch_list[0][0] -= 12
    for i in range(len(voices)):
#         voices[i][0] = pitch_list[0][roles[i]]+12*(random.randint(4, 6))
        voices[i][0] = pitch_list[0][i]+12*4
    print(voices)
    
    print(chord_choices)
#     for i in range(1, len(pitch_list)):
#         for j in range(len(voices)):
#             distances = []
#             for k in range(len(pitch_list[i])-1):
#                 distances.append((abs(pitch_list[i][k] - voices[j][i-1]%12), ((voices[j][i-1]//12)*12+pitch_list[i][k]) * random.choice([-1, 1]), ((voices[j][i-1]//12+1)*12+pitch_list[i][k]) * random.choice([-1, 1])))
#             distances.sort()
#             voices[j][i] = abs(distances[0][1]) if abs(voices[j][i-1] - abs(distances[0][1])) < abs(voices[j][i-1] - abs(distances[0][2])) else abs(distances[0][2])

    for i in range(1, len(pitch_list)):
        distances = []
        for j in range(len(pitch_list[i])):
            pitch_list[i].append(pitch_list[i].pop(0))
            if len(pitch_list[i]) == 3:
                pitch_list[i].append(pitch_list[i][0])
            
            distance = 0
            destination = []
            for k in range(len(voices)):
                delta = abs(pitch_list[i][k] - voices[k][i-1]%12)
                distance += delta
                lowOctave = (voices[j][i-1]//12)*12+pitch_list[i][k]
                highOctave = (voices[j][i-1]//12)*12+pitch_list[i][k]
                
                destination.append(lowOctave if abs(lowOctave - voices[k][i-1]) == delta else highOctave)
            distances.append((distance, tuple(destination)))
            
            if len(pitch_list[i]) == 3:
                pitch_list[i].pop()
                    
        distances.sort()
        for j in range(len(voices)):
            voices[j][i] = distances[0][1][j]
        
    output_midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=pretty_midi.instrument_name_to_program('Acoustic Grand Piano'))
    
    current_start = 0
    for i in range(1, len(voices[0])):
        same = True
        for j in range(len(voices)):
            if voices[j][current_start] != voices[j][i]:
                same = False
                break
        if not same:
            for j in range(len(voices)):
                instrument.notes.append(pretty_midi.Note(velocity=60, pitch=voices[j][current_start], start=beats[current_start], end=beats[i]))
            current_start = i
    for j in range(len(voices)):
        instrument.notes.append(pretty_midi.Note(velocity=60, pitch=voices[j][current_start], start=beats[current_start], end=beats[-1]+beats[1]-beats[0]))
#     for i in range(len(voices)):
#         for j in range(len(voices[i])):
#             instrument.notes.append(pretty_midi.Note(velocity=60, pitch=voices[i][j], start=beats[j], end=beats[j]+beats[1]-beats[0]))
    
    output_midi.instruments.append(instrument)
    output_midi.write('output_midi.mid')
    
get_accompaniment('eharmonicminormelody.mid', 2, 0)
    
    

[[47, 0, 0, 0, 0, 0, 0, 0], [50, 0, 0, 0, 0, 0, 0, 0], [54, 0, 0, 0, 0, 0, 0, 0], [59, 0, 0, 0, 0, 0, 0, 0]]
['di5', 'di5', 'di5', 'di5', 'di4', 'di4', 'di4', 'di4']
